In [133]:
# You need to replace these placeholders with your keys

consumer_key = 'qhSwqxpKslaBZofulQi598DrI'
consumer_secret = 'UTdmFpZb24IHcGc8CAaxArwq6LfyVMgUZz1gSuJuHyu7lj7hD0'
access_token = '793618519-K9kFIsgzD9vhp14kD5GxOLnGFEGl3jnjVZnwcMh5'
access_token_secret = 'ySEfCVU4qhRHZ8dTC7r1geQjFABInO0r79MWlK6TZxzLB'

In [134]:
# importing libraries
import pandas as pd
import requests
import tweepy
import numpy as np
import time
import json
import os
import contextlib
import warnings
import re

## Gathering data
---

In [135]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [136]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [137]:
# retrieve image predictions
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

# request from the url
req = requests.get(url)
if req.status_code == requests.codes.ok:
    req.encoding = 'utf-u'
    with open('image_predictions.txt', 'w') as infile:
        # write the text into the txt file
        infile.write(req.text)
        
# read the content using pandas 
df_image_predictions = pd.read_csv("image_predictions.txt", sep= '\t')

df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [138]:
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [139]:
# get the tweet IDs from df_archive and df_image_predictions
tweet_IDs = np.array(df_archive['tweet_id'].values, dtype = str)
tweet_IDs = np.concatenate((np.array(df_image_predictions['tweet_id'].values, dtype = str), tweet_IDs))
tweet_IDs = np.unique(tweet_IDs)
tweet_IDs

array(['666020888022790149', '666029285002620928', '666033412701032449',
       ..., '891815181378084864', '892177421306343426',
       '892420643555336193'],
      dtype='<U21')

In [140]:
#create the api instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

# create the lists where we will store the results from the twitter api
tweets = []
errors = []

for tweet_id in tweet_IDs:
    start = time.time()
    try:
        result = api.get_status(tweet_id, tweet_mode = 'extended')
        # Uncomment the lines below to see a JSON dump of one result
        # print(json.dumps(result, indent = 2))
        # break
        
        favorites = result['favorite_count']
        retweets = result['retweet_count']
        
        tweets.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets)})
    
    except Exception as e:
        errors.append(tweet_id)
    end = time.time()
    request_time = end - start
    print("time required: ", request_time)

time required:  0.5572190284729004
time required:  0.5420949459075928
time required:  0.5328519344329834
time required:  0.5303680896759033
time required:  0.26169800758361816
time required:  0.2624092102050781
time required:  0.33092308044433594
time required:  0.31145811080932617
time required:  0.2553250789642334
time required:  0.25579023361206055
time required:  0.24506115913391113
time required:  0.25469088554382324
time required:  0.30602025985717773
time required:  0.2473757266998291
time required:  0.26537394523620605
time required:  0.25109028816223145
time required:  0.29300713539123535
time required:  0.28612613677978516
time required:  0.2561941146850586
time required:  0.2608659267425537
time required:  0.28472089767456055
time required:  0.25688791275024414
time required:  0.2567720413208008
time required:  0.26458287239074707
time required:  0.2696549892425537
time required:  0.306751012802124
time required:  0.2562601566314697
time required:  0.24544906616210938
time r

time required:  0.2529747486114502
time required:  0.24941587448120117
time required:  0.2820448875427246
time required:  0.3063211441040039
time required:  0.2542273998260498
time required:  0.2656271457672119
time required:  0.2630021572113037
time required:  0.2618880271911621
time required:  0.312575101852417
time required:  0.2615339756011963
time required:  0.2611057758331299
time required:  0.2640650272369385
time required:  0.26617884635925293
time required:  0.2587709426879883
time required:  0.29765772819519043
time required:  0.29838991165161133
time required:  0.26521778106689453
time required:  0.2623937129974365
time required:  0.26349306106567383
time required:  0.26344990730285645
time required:  0.24929189682006836
time required:  0.30613231658935547
time required:  0.268934965133667
time required:  0.27658700942993164
time required:  0.2554817199707031
time required:  0.2731497287750244
time required:  0.31555891036987305
time required:  0.3042311668395996
time requir

time required:  0.252000093460083
time required:  0.2552359104156494
time required:  0.2496020793914795
time required:  0.2576868534088135
time required:  0.24782633781433105
time required:  0.30045270919799805
time required:  0.259915828704834
time required:  0.25343894958496094
time required:  0.2585477828979492
time required:  0.25879502296447754
time required:  0.2624969482421875
time required:  0.2966489791870117
time required:  0.2534818649291992
time required:  0.2940497398376465
time required:  0.32141733169555664
time required:  0.2886378765106201
time required:  0.30512213706970215
time required:  0.26648497581481934
time required:  0.2733268737792969
time required:  0.26050734519958496
time required:  0.2662992477416992
time required:  0.27121591567993164
time required:  0.27067995071411133
time required:  0.293438196182251
time required:  0.2572009563446045
time required:  0.2778630256652832
time required:  0.25133419036865234
time required:  0.26267290115356445
time requir

time required:  0.2839319705963135
time required:  0.25913286209106445
time required:  0.2501060962677002
time required:  0.2671539783477783
time required:  0.25386595726013184
time required:  0.25272297859191895
time required:  0.25083303451538086
time required:  0.4055182933807373
time required:  0.26456189155578613
time required:  0.25511717796325684
time required:  0.2971377372741699
time required:  0.25834012031555176
time required:  0.25348806381225586
time required:  0.30420804023742676
time required:  0.2520003318786621
time required:  0.25709080696105957
time required:  0.2509150505065918
time required:  0.27086305618286133
time required:  0.26209473609924316
time required:  0.2692439556121826
time required:  0.254575252532959
time required:  0.2706577777862549
time required:  0.28127598762512207
time required:  0.30426502227783203
time required:  0.2501828670501709
time required:  0.3269529342651367
time required:  0.2821769714355469
time required:  0.29145288467407227
time r

time required:  0.25996971130371094
time required:  0.25775694847106934
time required:  0.26485705375671387
time required:  0.29253292083740234
time required:  0.254763126373291
time required:  0.30091285705566406
time required:  0.2561519145965576
time required:  0.2627990245819092
time required:  0.2590198516845703
time required:  0.2944929599761963
time required:  0.26360487937927246
time required:  0.26337099075317383
time required:  0.2654080390930176
time required:  0.2532010078430176
time required:  0.2577962875366211
time required:  0.27122020721435547
time required:  0.29549384117126465
time required:  0.2716639041900635
time required:  0.2812938690185547
time required:  0.25943899154663086
time required:  0.25443506240844727
time required:  0.28057074546813965
time required:  0.2892720699310303
time required:  0.2575199604034424
time required:  0.3437941074371338
time required:  0.2862589359283447
time required:  0.2874753475189209
time required:  0.2659330368041992
time requ

...

time required:  0.2496650218963623
time required:  0.26253485679626465
time required:  0.28650379180908203
time required:  0.29735612869262695
time required:  0.26008009910583496
time required:  0.25643301010131836
time required:  0.26293206214904785
time required:  0.2583918571472168
time required:  0.2813150882720947
time required:  0.26886796951293945
time required:  0.2591869831085205
time required:  0.25585103034973145
time required:  0.2580881118774414
time required:  0.2795860767364502
time required:  0.3244960308074951
time required:  0.26689577102661133
time required:  0.25522494316101074
time required:  0.2837657928466797
time required:  0.26867127418518066
time required:  0.2668890953063965
time required:  0.2738230228424072
time required:  0.2873110771179199
time required:  0.2498481273651123
time required:  0.2978806495666504
time required:  0.2602670192718506
time required:  0.27385497093200684
time required:  0.2719419002532959
time required:  0.2666819095611572
time req

time required:  0.30582094192504883
time required:  0.2653770446777344
time required:  0.25725603103637695
time required:  0.26356983184814453
time required:  0.29592394828796387
time required:  0.2704460620880127
time required:  0.2627298831939697
time required:  0.25764918327331543
time required:  0.25484180450439453
time required:  0.2562448978424072
time required:  0.24936389923095703
time required:  0.3250901699066162
time required:  0.26018381118774414
time required:  0.2930920124053955
time required:  0.2677500247955322
time required:  0.30028605461120605
time required:  0.2713911533355713
time required:  0.24870705604553223
time required:  0.2548179626464844
time required:  0.2751491069793701
time required:  0.30532193183898926
time required:  0.2771339416503906
time required:  0.3138120174407959
time required:  0.25293397903442383
time required:  0.26180005073547363
time required:  0.25701117515563965
time required:  0.26414990425109863
time required:  0.2878000736236572
time 

time required:  0.2600090503692627
time required:  0.2926790714263916
time required:  0.30893588066101074
time required:  0.2745389938354492
time required:  0.3167428970336914
time required:  0.2615017890930176
time required:  0.2510659694671631
time required:  0.3090670108795166
time required:  0.2593669891357422
time required:  0.2568361759185791
time required:  0.2630038261413574
time required:  0.2500159740447998
time required:  0.25986790657043457
time required:  0.25264906883239746
time required:  0.25863099098205566
time required:  0.30184388160705566
time required:  0.2775700092315674
time required:  0.2578113079071045
time required:  0.2488551139831543
time required:  0.26265501976013184
time required:  0.3317737579345703
time required:  0.24858593940734863
time required:  0.5351090431213379
time required:  0.25647687911987305
time required:  0.2501027584075928
time required:  0.2908768653869629
time required:  0.27623891830444336
time required:  0.2627220153808594
time requir

time required:  0.2745239734649658
time required:  0.2554652690887451
time required:  0.28489208221435547
time required:  0.26091909408569336
time required:  0.28646016120910645
time required:  0.3160209655761719
time required:  0.3246159553527832
time required:  0.24858713150024414
time required:  0.2674827575683594
time required:  0.24160408973693848
time required:  0.25409626960754395
time required:  0.3200037479400635
time required:  0.5209681987762451
time required:  0.2726140022277832
time required:  0.3576192855834961
time required:  0.27928709983825684
time required:  0.28040075302124023
time required:  0.2577948570251465
time required:  0.2600722312927246
time required:  0.2851898670196533
time required:  0.26607418060302734
time required:  0.25172924995422363
time required:  0.2701380252838135
time required:  0.27679991722106934
time required:  0.26055097579956055
time required:  0.34754514694213867
time required:  0.26812219619750977
time required:  0.2582368850708008
time r

time required:  0.2606790065765381
time required:  0.2794370651245117
time required:  0.3573009967803955
time required:  0.302534818649292
time required:  0.26500988006591797
time required:  0.285567045211792
time required:  0.6882209777832031
time required:  0.2720680236816406
time required:  0.24886107444763184
time required:  0.7093009948730469
time required:  0.2704761028289795
time required:  0.2560861110687256
time required:  0.3091700077056885
time required:  0.3005061149597168
time required:  0.2577190399169922
time required:  0.2503960132598877
time required:  0.25373101234436035
time required:  0.29973912239074707
time required:  0.2309436798095703
time required:  0.2612152099609375
time required:  0.2799980640411377
time required:  0.26298999786376953
time required:  0.2905731201171875
time required:  0.27167582511901855
time required:  0.2723400592803955
time required:  0.8459310531616211
time required:  0.2651808261871338
time required:  0.2725808620452881
time required:  

In [141]:
print("The number of results from twitter", len(tweets))
print("The number of errors", len(errors))

The number of results from twitter 2347
The number of errors 9


In [142]:
with contextlib.suppress(FileNotFoundError):
    os.remove('tweet_json.txt')

df_json_tweets = pd.DataFrame(tweets, columns = ['tweet_id', 'favorites', 'retweets'])

df_json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)
df_json_tweets.head()

,tweet_id,favorites,retweets
0,666020888022790149,2530,523
1,666029285002620928,131,47
2,666033412701032449,126,46
3,666044226329800704,305,143
4,666049248165822465,110,40


## Assesing data
---

### Visual inspection

In [143]:
df_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None
9,89024025534

In [144]:
df_image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


Observation: p1_conf > p2conf > p3_conf. Therefor it is not needed to keep p2, p2_conf, p2_dog, p3, p3_conf, p3_dog
Observation 2: Some pictures are not of dogs i.e. https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg identified by the neural network as hen shows indeed a hen.
Obervation 3: Some picture identified as not a dog are in fact dogs i.e. https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg identified as a shopping cart shows a dog sitting in a shopping cart.
        

In [145]:
df_json_tweets

,tweet_id,favorites,retweets
0,666020888022790149,2530,523
1,666029285002620928,131,47
2,666033412701032449,126,46
3,666044226329800704,305,143
4,666049248165822465,110,40
5,666050758794694657,135,59
6,666051853826850816,1241,868
7,666055525042405380,446,259
8,666057090499244032,303,144
9,666058600524156928,115,59


I do not notice anything remarkable in this table. 

### Programmatic inspection

In [146]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Observation: many variables have the wrong type i.e. object should be string for name source and text. 

In [147]:
df_archive.nunique()

tweet_id                      2356
in_reply_to_status_id         77  
in_reply_to_user_id           31  
timestamp                     2356
source                        4   
text                          2356
retweeted_status_id           181 
retweeted_status_user_id      25  
retweeted_status_timestamp    181 
expanded_urls                 2218
rating_numerator              40  
rating_denominator            18  
name                          957 
doggo                         2   
floofer                       2   
pupper                        2   
puppo                         2   
dtype: int64

In [148]:
df_archive.source.unique()

array([ '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'], dtype=object)

Observation: Dirty. As observed before the sources are hard to read. As there only 4 different sources this is easy to fix with search and replace.

In [149]:
df_archive.doggo.unique()

array(['None', 'doggo'], dtype=object)

Observation: This is a tidyness issue doggo, floofer, pupper, puppo should be in one variabele.

In [150]:
df_archive['name'].value_counts()

None         745
a            55 
Charlie      12 
Lucy         11 
Oliver       11 
Cooper       11 
Tucker       10 
Penny        10 
Lola         10 
Bo           9  
Winston      9  
the          8  
Sadie        8  
Toby         7  
Buddy        7  
Daisy        7  
an           7  
Bailey       7  
Oscar        6  
Koda         6  
Scout        6  
Leo          6  
Bella        6  
Rusty        6  
Jack         6  
Milo         6  
Dave         6  
Stanley      6  
Jax          6  
Alfie        5  
            ..  
Aja          1  
Stu          1  
Dotsy        1  
Bobbay       1  
Hubertson    1  
Peanut       1  
Billy        1  
Mack         1  
Kellogg      1  
Billl        1  
Rupert       1  
Craig        1  
Zeus         1  
Travis       1  
Spencer      1  
Dudley       1  
Liam         1  
Flurpson     1  
Brian        1  
Timmy        1  
Pete         1  
Florence     1  
Glenn        1  
Jockson      1  
Mary         1  
Petrick      1  
all          1  
Jangle       1

This is a dirty data issue. Names like "None", "a" , "an", and "the" are not likely dog names. 

In [151]:
df_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7       55 
14      54 
5       37 
6       32 
3       19 
4       17 
1       9  
2       9  
420     2  
0       2  
15      2  
75      2  
80      1  
20      1  
24      1  
26      1  
44      1  
50      1  
60      1  
165     1  
84      1  
88      1  
144     1  
182     1  
143     1  
666     1  
960     1  
1776    1  
17      1  
27      1  
45      1  
99      1  
121     1  
204     1  
Name: rating_numerator, dtype: int64

Observation: As mentioned in the assignment many numerators are above 10. This is not an issue.

In [152]:
df_archive['rating_denominator'].value_counts()

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

Observation: Most keep the denominator on 10. A few choose another scale. Hmm I don't trust this. Time for an extra visual inspection.

In [153]:
pd.set_option('display.max_colwidth', -1)
df_not_ten = df_archive[df_archive['rating_denominator'] != 10]
df_not_ten[['tweet_id', 'text','rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
313,835246439529840640,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
342,832088576586297345,@docmisterio account started on 11/15/15,11,15
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
784,775096608509886464,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50


Observation: when the denominator is not 10 it is often caused by other fraction being misidentified as the rating i.e 7/11 the store or 3 1/2 legged or 11/15/15 the date.

In [154]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Observation: The 2075 records found here is below the records found for the df_archive dataset. Meaning that there are tweets without an image.

In [155]:
df_image_predictions.nunique()

tweet_id    2075
jpg_url     2009
img_num     4   
p1          378 
p1_conf     2006
p1_dog      2   
p2          405 
p2_conf     2004
p2_dog      2   
p3          408 
p3_conf     2006
p3_dog      2   
dtype: int64

In [156]:
df_image_predictions['jpg_url'].value_counts()

https://pbs.twimg.com/media/C3nygbBWQAAjwcW.jpg                                            2
https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg                                            2
https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg                                            2
https://pbs.twimg.com/tweet_video_thumb/CeBym7oXEAEWbEg.jpg                                2
https://pbs.twimg.com/media/CxqsX-8XUAAEvjD.jpg                                            2
https://pbs.twimg.com/media/CpmyNumW8AAAJGj.jpg                                            2
https://pbs.twimg.com/media/Cx5R8wPVEAALa9r.jpg                                            2
https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg                                            2
https://pbs.twimg.com/media/CVuQ2LeUsAAIe3s.jpg                                            2
https://pbs.twimg.com/media/CiyHLocU4AI2pJu.jpg                                            2
https://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg                       

In [157]:
df_json_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2347 entries, 0 to 2346
Data columns (total 3 columns):
tweet_id     2347 non-null int64
favorites    2347 non-null int64
retweets     2347 non-null int64
dtypes: int64(3)
memory usage: 55.1 KB


In [158]:
# this returns the number of unique values in each column
df_json_tweets.nunique()

tweet_id     2347
favorites    2017
retweets     1741
dtype: int64

### Tidiness issues (structual issues)
* To much information in df_image_predictions dataframe, (tweet_id and jpg_url is what we need)
* Various stages of dogs (doggo, floofer, pupper, puppo) are in four columns.
* All dataframes should be merged into one.

### Dirtiness issues (quality)
* wrong datatypes for several columns
* Tweets without images
* Dataset contains retweets
* Incorrect dog names
* Sources difficult to read
* Some tweet_ids have the same jpg_url
* Some pictures are not of dogs
* Some pictures are flagged as not a dog but are in fact of a dog.
* Fractions in the text are mistaken for rates.

## Cleaning data
---

### Creating copies of dataframes for cleaning

In [159]:
df_archive_clean = df_archive.copy()
df_image_predictions_clean = df_image_predictions.copy()
df_json_tweets_clean = df_json_tweets.copy()

### Define
The source column should be a category with four elements: "Twitter for iPhone", "Vine - Make a Scene", "Twitter Web Client", "TweetDeck" . In this step I remove the HTML tags using a simple regular expression and change the datatype to category.

### Code

In [160]:
df_archive_clean['source'] = df_archive_clean['source'].str.extract('>(.*?)</a>', expand= True)
df_archive_clean = df_archive_clean.astype({'source': "category"})

### Test

In [161]:
df_archive_clean['source'].value_counts()

Twitter for iPhone     2221
Vine - Make a Scene    91  
Twitter Web Client     33  
TweetDeck              11  
Name: source, dtype: int64

### Define
Create a single dog stage column and remove the four individual dog stage columns.

### Code

In [162]:
df_archive_clean['dog_stage'] = df_archive_clean['text'].str.extract('(doggo|floofer|pupper|puppo)', expand=True)
columns = ['doggo', 'floofer', 'pupper', 'puppo']
df_archive_clean = df_archive_clean.drop(columns, axis=1)

### Test

In [163]:
# Shows that there is a new column and the 4 original stage columns are removed
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null category
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
dog_stage                     399 non-null object
dtypes: category(1), float64(4), int64(3), object(6)
memory usage: 241.8+ KB


In [164]:
df_archive_clean['dog_stage'].value_counts()

pupper     265
doggo      93 
puppo      37 
floofer    4  
Name: dog_stage, dtype: int64

### Define
Merge df_image_predictions_clean and df_json_tweets_clean into df_archive_clean.

### Code

In [165]:
df_archive_clean = df_archive_clean.merge(df_image_predictions_clean, on='tweet_id', how='inner')
df_archive_clean = df_archive_clean.merge(df_json_tweets_clean, on='tweet_id', how='inner')

### Test

In [166]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 0 to 2068
Data columns (total 27 columns):
tweet_id                      2069 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2069 non-null object
source                        2069 non-null category
text                          2069 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2069 non-null object
rating_numerator              2069 non-null int64
rating_denominator            2069 non-null int64
name                          2069 non-null object
dog_stage                     337 non-null object
jpg_url                       2069 non-null object
img_num                       2069 non-null int64
p1                            2069 non-null object
p1_conf                       2069 

### Define
We want the original tweets. That means that we should only keep rows where retweeted_status_user_id and in_reply_to_user_id are null.

### Code

In [167]:
df_archive_clean = df_archive_clean[df_archive_clean['retweeted_status_id'].isnull()]
df_archive_clean = df_archive_clean[df_archive_clean['in_reply_to_user_id'].isnull()]

### Test

In [168]:
df_archive_clean[df_archive_clean['retweeted_status_id'].notnull()].shape[0]

0

In [169]:
df_archive_clean[df_archive_clean['in_reply_to_user_id'].notnull()].shape[0]

0

### Define
Now that the retweets are removed all the columns related to retweets and replies can be removed also.

### Code

In [170]:
drop_columns = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 
                'in_reply_to_user_id', 'in_reply_to_status_id']
df_archive_clean = df_archive_clean.drop(drop_columns, axis=1)

### Test

In [171]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2068
Data columns (total 22 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null object
source                1971 non-null category
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
dog_stage             322 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null bool
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog                1971 non-null bool
favorites             1971 non-null int

### Define
Records without images should be removed.

### Code

In [172]:
df_archive_clean = df_archive_clean.dropna(subset=['expanded_urls'])

### Test

In [173]:
df_archive_clean[df_archive_clean['expanded_urls'].isnull()].shape[0]

0

### Define 
For the image predictions I only want to keep the one with the greatest confidence.

### Code

In [174]:
# First make sure p1_conf is always larger than p2_conf 
#(this might be obvious for a machine learning engineer but machine learning is my next nanodegree)
df_archive_clean[df_archive_clean['p1_conf'] < df_archive_clean['p2_conf']].shape[0]

0

In [175]:
# And make sure p1_conf is always larger than p3_conf
df_archive_clean[df_archive_clean['p1_conf'] < df_archive_clean['p3_conf']].shape[0]

0

In [176]:
# p1_conf is always greater than p2_conf and p3_conf so we can drop everything related to p2 and p3
drop_columns = ['p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog']
df_archive_clean = df_archive_clean.drop(drop_columns, axis=1)

### Test

In [177]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2068
Data columns (total 16 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null object
source                1971 non-null category
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
dog_stage             322 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
favorites             1971 non-null int64
retweets              1971 non-null int64
dtypes: bool(1), category(1), float64(1), int64(6), object(7)
memory usage: 235.0+ KB


### Define
Change columns to the correct datatype.

### Code

In [178]:
df_archive_clean['dog_stage'] = df_archive_clean['dog_stage'].astype('category')
df_archive_clean['timestamp'] = pd.to_datetime(df_archive_clean['timestamp'])
df_archive_clean['tweet_id'] = df_archive_clean['tweet_id'].astype('str')
df_archive_clean['name'] = df_archive_clean['name'].astype('str')

### Test

In [179]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2068
Data columns (total 16 columns):
tweet_id              1971 non-null object
timestamp             1971 non-null datetime64[ns]
source                1971 non-null category
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
dog_stage             322 non-null category
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
favorites             1971 non-null int64
retweets              1971 non-null int64
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 221.7+ KB


### Define
Fix the ratings that are not really ratings.

### Code

In [180]:
# First lets get an overview of the numerators
df_archive_clean.rating_numerator.value_counts()

12      446
10      417
11      392
13      253
9       150
8       95 
7       52 
5       33 
14      33 
6       32 
3       19 
4       16 
2       9  
1       5  
1776    1  
24      1  
26      1  
420     1  
27      1  
44      1  
45      1  
50      1  
60      1  
75      1  
80      1  
84      1  
88      1  
99      1  
121     1  
144     1  
165     1  
204     1  
0       1  
Name: rating_numerator, dtype: int64

In [181]:
# And the denominators
df_archive_clean.rating_denominator.value_counts()

10     1954
50     3   
80     2   
11     2   
170    1   
150    1   
120    1   
110    1   
90     1   
70     1   
40     1   
20     1   
7      1   
2      1   
Name: rating_denominator, dtype: int64

In [182]:
# Find the tweets with more than one %d/%d in the 'text' column. With %d being any number.
warnings.filterwarnings("ignore", 'This pattern has match groups')
df_archive_clean[df_archive_clean.text.str.contains( r"(\d+\/\d+\D+\d+\/\d+)")]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,jpg_url,img_num,p1,p1_conf,p1_dog,favorites,retweets
612,777684233540206592,2016-09-19 01:42:24,Twitter for iPhone,"""Yep... just as I suspected. You're not flossing."" 12/10 and 11/10 for the pup not flossing https://t.co/SuXcI9B7pQ",https://twitter.com/dog_rates/status/777684233540206592/photo/1,12,10,None,NaN,https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg,1,cocker_spaniel,0.253442,True,12376,3349
821,747600769478692864,2016-06-28 01:21:27,Twitter for iPhone,This is Bookstore and Seaweed. Bookstore is tired and Seaweed is an asshole. 10/10 and 7/10 respectively https://t.co/eUGjGjjFVJ,"https://twitter.com/dog_rates/status/747600769478692864/photo/1,https://twitter.com/dog_rates/status/747600769478692864/photo/1",10,10,Bookstore,NaN,https://pbs.twimg.com/media/CmAC7ehXEAAqSuW.jpg,1,Chesapeake_Bay_retriever,0.804363,True,2526,611
872,740373189193256964,2016-06-08 02:41:38,Twitter for iPhone,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,NaN,https://pbs.twimg.com/media/CkZVdJ6WYAAXZ5A.jpg,3,golden_retriever,0.807644,True,37563,14854
963,722974582966214656,2016-04-21 02:25:47,Twitter for iPhone,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,None,NaN,https://pbs.twimg.com/media/CgiFjIpWgAA4wVp.jpg,1,Great_Dane,0.246762,True,4433,1735
997,716439118184652801,2016-04-03 01:36:11,Twitter for iPhone,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,https://twitter.com/dog_rates/status/716439118184652801/photo/1,50,50,Bluebert,NaN,https://pbs.twimg.com/media/CfFNk7cWAAA-hND.jpg,1,Siberian_husky,0.396495,True,2545,243
1013,714258258790387713,2016-03-28 01:10:13,Twitter for iPhone,Meet Travis and Flurp. Travis is pretty chill but Flurp can't lie down properly. 10/10 &amp; 8/10\nget it together Flurp https://t.co/Akzl5ynMmE,https://twitter.com/dog_rates/status/714258258790387713/photo/1,10,10,Travis,NaN,https://pbs.twimg.com/media/CemOGNjWQAEoN7R.jpg,1,collie,0.176758,True,3240,798
1135,703356393781329922,2016-02-26 23:10:06,Twitter for iPhone,This is Socks. That water pup w the super legs just splashed him. Socks did not appreciate that. 9/10 and 2/10 https://t.co/8rc5I22bBf,https://twitter.com/dog_rates/status/703356393781329922/photo/1,9,10,Socks,NaN,https://pbs.twimg.com/media/CcLS6QKUcAAUuPa.jpg,1,Border_collie,0.894842,True,2058,426
1225,695064344191721472,2016-02-04 02:00:27,Twitter for iPhone,"This may be the greatest video I've ever been sent. 4/10 for Charles the puppy, 13/10 overall. (Vid by @stevenxx_) https://t.co/uaJmNgXR2P",https://twitter.com/dog_rates/status/695064344191721472/video/1,4,10,None,NaN,https://pbs.twimg.com/ext_tw_video_thumb/695064251149508610/pu/img/0OPED0aUurb9Z16a.jpg,1,seat_belt,0.522211,False,1747,673
1230,694352839993344000,2016-02-02 02:53:12,Twitter for iPhone,Meet Oliviér. He takes killer selfies. Has a dog of his own. It leaps at random &amp; can't bark for shit. 10/10 &amp; 5/10 https://t.co/6NgsQJuSBJ,"https://twitter.com/dog_rates/status/694352839993344000/photo/1,https://twitter.com/dog_rates/status/694352839993344000/photo/1,https://twitter.com/dog_rates/status/694352839993344000/photo/1,https://twitter.com/dog_rates/status/694352839993344000/photo/1",10,10,Oliviér,NaN,https://pbs.twimg.com/media/CaLWOPfWkAAo2Dt.jpg,2,Australian_terrier,0.407886,True,2206,691
1265,691483041324204033,2016-01-25 04:49:38,Twitter for iPhone,When bae says they can't go out but you see them with someone else t

Observations:
TweetId 740373189193256964: Rating received 9/11 should be 14/10
TweetId 722974582966214656: Rating received 4/20 should be 13/10
TweetId 716439118184652801: Rating received 50/50 should be 11/10
TweetId 682962037429899265: Rating received 7/11 should be 10/10
TweetId 666287406224695296: Rating received 1/2 should be 9/10
The other cases are caused by multiple dogs or a dog and another creature. In this case I kept the first one.

In [183]:
def set_fraction(tweet_id, numerator, denominator):
    target_index = df_archive_clean.loc[df_archive_clean['tweet_id']==tweet_id].index[0]
    df_archive_clean.loc[target_index, 'rating_numerator'] = numerator
    df_archive_clean.loc[target_index, 'rating_denominator'] = denominator

set_fraction('740373189193256964', 14, 10)
set_fraction('722974582966214656', 13, 10)
set_fraction('716439118184652801', 11, 10)
set_fraction('682962037429899265', 10, 10)
set_fraction('666287406224695296', 9, 10)

### Test

In [184]:
changed_tweets = ['740373189193256964', '722974582966214656', 
                  '716439118184652801', '682962037429899265', '666287406224695296']
df_archive_clean.loc[df_archive_clean['tweet_id'].isin(changed_tweets), 
                     ['tweet_id', 'rating_numerator', 'rating_denominator']]

,tweet_id,rating_numerator,rating_denominator
872,740373189193256964,14,10
963,722974582966214656,13,10
997,716439118184652801,11,10
1401,682962037429899265,10,10
2048,666287406224695296,9,10


### Define
Fix the invalid dog names.

In [185]:
# Before we move to the coding let us first have a visual glance. For starters dogname 'None'is invalid.
df_none = df_archive_clean[df_archive_clean.name == 'None']
df_none['text']

5       Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh   
7       When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq                       
12      Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm                                   
23      You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV                                                     
24      This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp            
33      Here we have a corgi undercover as a malamute. Pawbably doing important investigative work. Zero control over tongue happe

In [186]:
# some more invalid names
df_lower_case = df_archive_clean[df_archive_clean.name.str.islower()]
df_lower_case['text']

21      I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba                              
50      Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF       
139     We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10 https://t.co/g2nSyGenG9              
153     Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective https://t.co/Xc7uj1C64x
290     Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK                                                          
435     We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10 https://t.co/

In [187]:
# some more invalid names
df_one_letter = df_archive_clean[df_archive_clean.name.str.len() == 1]
df_one_letter['text']

50      Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF
520     Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq                           
620     This is O'Malley. That is how he sleeps. Doesn't care what you think about it. 10/10 comfy af https://t.co/Pq150LeRaC                                      
643     Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn               
818     This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW               
820     Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R                 
829     This is 

### Observations
Many texts simply have no dog names. So we can't resolve all
Dogs are often introduced with "This is Dogname", "Meet Dogname", "Their names are Dogname1 and Dogname2", "Her name is Dogname", "named Dogname", "His name is Dogname"
Note that we considder only dognames that are cappitalized. So "This is Fred" is a match and "This is the" is not.

### Code

In [188]:
# Rename the old 'name' colmn to old_name
df_archive_clean.rename(columns = {'name':'old_name'}, inplace = True)
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2068
Data columns (total 16 columns):
tweet_id              1971 non-null object
timestamp             1971 non-null datetime64[ns]
source                1971 non-null category
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
old_name              1971 non-null object
dog_stage             322 non-null category
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
favorites             1971 non-null int64
retweets              1971 non-null int64
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 221.7+ KB


In [189]:
names = []

for text in df_archive_clean['text']:
    # Start with 'This is ' and the first letter of the name is uppercase
    if text.startswith('This is ') and re.match(r'[A-Z].*', text.split()[2]):
        names.append(text.split()[2].strip(',').strip('.'))
    # Start with 'Meet ' and the first letter of the name is uppercase
    elif text.startswith('Meet ') and re.match(r'[A-Z].*', text.split()[1]):
        names.append(text.split()[1].strip(',').strip('.'))
    # Contain 'names are' and the first letter of the name is uppercase
    elif 'names are' in text and re.match(r'[A-Z].*', text.split()[text.split().index('names') + 2]):
        names.append(text.split()[text.split().index('names') + 2].strip(',').strip('.'))
    # Contain 'name is' and the first letter of the name is uppercase
    elif 'name is' in text and re.match(r'[A-Z].*', text.split()[text.split().index('name') + 2]):
        names.append(text.split()[text.split().index('name') + 2].strip(',').strip('.'))
    # Contain 'named' and the first letter of the name is uppercase
    elif 'named' in text and re.match(r'[A-Z].*', text.split()[text.split().index('named') + 1]):
        names.append(text.split()[text.split().index('named') + 1].strip(',').strip('.'))
    else:
        names.append('None')
        
df_archive_clean['name'] = names

In [190]:
print("New column dog_name count \n", df_archive_clean.name.value_counts())
print("Old column name count \n", df_archive_clean.old_name.value_counts())

New column dog_name count 
 None         689
Charlie      10 
Lucy         9  
Cooper       9  
Tucker       9  
Penny        8  
Oliver       8  
Lola         7  
Winston      7  
Daisy        7  
Jax          6  
Stanley      6  
Sadie        6  
Bo           6  
Bella        6  
Koda         6  
Toby         6  
Rusty        5  
Bailey       5  
Buddy        5  
Zoey         5  
Chester      5  
Milo         5  
Louis        5  
Oscar        5  
Scout        5  
Dave         4  
Gary         4  
Phil         4  
Alfie        4  
            ..  
Cecil        1  
Bruiser      1  
Brudge       1  
Vixen        1  
Crouton      1  
Banjo        1  
Henry        1  
Kathmandu    1  
Fizz         1  
Iggy         1  
Puff         1  
Bayley       1  
Harper       1  
Jameson      1  
Strudel      1  
Barclay      1  
Obi          1  
Pipsy        1  
Karll        1  
Philbert     1  
Eleanor      1  
Alf          1  
Vince        1  
Rumpole      1  
Gustav       1  
Arya         1  
Eri

In [191]:
# remove the old names column
del df_archive_clean['old_name']
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2068
Data columns (total 16 columns):
tweet_id              1971 non-null object
timestamp             1971 non-null datetime64[ns]
source                1971 non-null category
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
dog_stage             322 non-null category
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
favorites             1971 non-null int64
retweets              1971 non-null int64
name                  1971 non-null object
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 221.7+ KB


### Test

In [192]:
# Check for one letter names
df_one_letter = df_archive_clean[df_archive_clean.name.str.len() == 1]
df_one_letter

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,dog_stage,jpg_url,img_num,p1,p1_conf,p1_dog,favorites,retweets,name


In [193]:
# Check for lower case names
df_lower_case = df_archive_clean[df_archive_clean.name.str.islower()]
df_lower_case

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,dog_stage,jpg_url,img_num,p1,p1_conf,p1_dog,favorites,retweets,name


Note there are still tweets with name == 'None' simply because a lot of tweets did not give the dogs name.

## Storing the cleaned data

In [194]:
df_archive_clean.to_csv('twitter_archive_cleaned_and_shiny.csv', index=False, encoding = 'utf-8')

## Analysis
### Read in the data

In [195]:
df_analysis = pd.read_csv('twitter_archive_cleaned_and_shiny.csv')
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 16 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null object
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
dog_stage             322 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
favorites             1971 non-null int64
retweets              1971 non-null int64
name                  1971 non-null object
dtypes: bool(1), float64(1), int64(6), object(8)
memory usage: 233.0+ KB


In [196]:
df_analysis.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,dog_stage,jpg_url,img_num,p1,p1_conf,p1_dog,favorites,retweets,name
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,NaN,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,39193,8738,Phineas
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,NaN,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,33575,6412,Tilly
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,NaN,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,25306,4257,Archie
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,NaN,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,42561,8835,Darla
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,NaN,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,40750,9618,Franklin
